In [17]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
batchsize=1

In [5]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,
                              zoom_range=0.2, width_shift_range=0.2,
                              height_shift_range=0.2, validation_split=0.2)

train_data=train_datagen.flow_from_directory(r'D:\AI Project\Drowsiness detection Code\Drowsiness detection\Anotherdataset\Prepared_data\train'
                                            ,target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='training')

validation_data= train_datagen.flow_from_directory(r'D:\AI Project\Drowsiness detection Code\Drowsiness detection\Anotherdataset\Prepared_data\train'
                                            ,target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')


Found 64719 images belonging to 2 classes.
Found 16179 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\AI Project\Drowsiness detection Code\Drowsiness detection\Anotherdataset\Prepared_data\test'
                                            ,target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4000 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3), batch_size=batchsize))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs =bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False 

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(1, 80, 80, 3)]     0           []                               
                                                                                                  
 conv2d (Conv2D)                (1, 39, 39, 32)      864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (1, 39, 39, 32)     96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (1, 39, 39, 32)      0           ['batch_normalization[0][0]']

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint(r'D:\AI Project\Drowsiness detection Code\Drowsiness detection\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose=3)

callbacks = [checkpoint,earlystop,learning_rate]

In [ ]:

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                  validation_steps=validation_data.samples//batchsize,
                  callbacks=callbacks,
                   epochs=5)
#model.fit(train_data,epochs=5)


In [12]:
#Model Evalution

In [14]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

<ipython-input-14-87a1e491e135>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.17661353945732117
0.9333117008209229


In [15]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

<ipython-input-15-857f2865b528>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.22067299485206604
0.9125409722328186


In [16]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

<ipython-input-16-6b2bbd947a80>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.5294550061225891
0.8162500262260437
